In [2]:
import json
from time import time
import pandas as pd
from pathlib import Path
from dpmm.pipelines import MSTPipeline


/tmp/ipykernel_28183/2765868793.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Load data - HMEQ

In [ ]:
hmeq_dir = Path().parent / "hmeq"

dtypes = {
    "BAD": "category",
    "LOAN": "Int64",
    "MORTDUE": "float64",
    "VALUE": "float64",
    "REASON": "category",
    "JOB": "category",
    "YOJ": "float64",
    "DEROG": "category",
    "DELINQ": "category",
    "CLAGE": "float64",
    "NINQ": "category",
    "CLNO": "Int64",
    "DEBTINC": "float",
   }
df = pd.read_csv(hmeq_dir / "hmeq.csv", dtype=dtypes)

with (Path().parent / "hmeq" / "hmeq_bounds.json").open("r") as fr:
    domain = json.load(fr)


df.head()

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,1,1100,25860.0,39025.0,HomeImp,Other,10.5,0,0,94.366667,1,9,NaN
1,1,1300,70053.0,68400.0,HomeImp,Other,7.0,0,2,121.833333,0,14,NaN
2,1,1500,13500.0,16700.0,HomeImp,Other,4.0,0,0,149.466667,1,10,NaN
3,1,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN
4,0,1700,97800.0,112000.0,HomeImp,Office,3.0,0,0,93.333333,0,14,NaN


# Build pipeline

In [ ]:
pipeline = MSTPipeline(
    epsilon=1,  # Privacy budget for generate model
    proc_epsilon=0.1,  # Privacy budget for data processing 
    delta=1e-5,  # Delta Setting in (eps,  delta) differential privacy
)

# Fit step

In [19]:
print("Fitting MST pipeline...", end='\r')
start_time = time()
# The number of rows to generate
pipeline.fit(df, domain)
fit_time = time()

print(f"Fitting MST pipeline - Took {fit_time - start_time:.2f} seconds")

Fitting MST pipeline - Took 28.63 seconds


# Generate step

In [20]:
print("Generating synthetic data...", end="\r")
# Generate synthetic data
synth_df = pipeline.generate(df.shape[0])
gen_time = time()

print(f"Generating synthetic data - Took {gen_time - fit_time:.2f} seconds")

synth_df.head()

Generating synthetic data - Took 93.77 seconds


,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,0,13899,49028.492901,65482.680989,HomeImp,Other,9.876399,0,0,162.171895,0,21,37.438151
1,0,21754,43399.276566,66202.665966,NaN,Other,0.580768,NaN,0,NaN,0,21,37.833572
2,0,8685,65312.552999,56228.036231,DebtCon,Mgr,4.540470,0,0,195.813745,0,<NA>,36.569078
3,0,26325,325366.191208,84295.091562,DebtCon,Other,9.651415,0,0,160.743188,1,25,23.921458
4,1,28046,57733.563724,30179.169117,DebtCon,ProfExe,4.216379,0,2,218.274103,0,17,NaN
